In [1]:
import os
import sys


# IMPORTANT: Set these BEFORE importing torch or any ML libraries
# Disable all GPU/MPS backends to prevent crashes with IFD analyzer
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Disable CUDA
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"  # Disable MPS memory allocation
os.environ["DISABLE_MPS_COMPAT"] = "1"  # Additional MPS disable flag
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"  # Disable HuggingFace telemetry
os.environ["TRANSFORMERS_OFFLINE"] = "0"  # Allow model downloads

# Force CPU usage in PyTorch to avoid MPS crashes
import torch

# Forcefully disable MPS before anything else
torch.set_default_device("cpu")
if hasattr(torch.backends, "mps"):
    # Monkey-patch to prevent MPS usage
    original_is_available = torch.backends.mps.is_available
    torch.backends.mps.is_available = lambda: False
    print("MPS has been disabled - forcing CPU-only mode")
else:
    print("Using CPU for all computations")

print(f"PyTorch device: {torch.get_default_device()}")
print(f"PyTorch version: {torch.__version__}")

MPS has been disabled - forcing CPU-only mode
PyTorch device: cpu
PyTorch version: 2.6.0


In [2]:
import os
from oumi.core.configs import AnalyzeConfig
from oumi.core.analyze.dataset_analyzer import DatasetAnalyzer

# Path to the config file
config_path = "/Users/ryanarman/code/oumi/configs/examples/analyze/analyze.yaml"

# Path to your dataset file
dataset_path = "/Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl"

# Load the config from YAML
config = AnalyzeConfig.from_yaml(
    config_path=config_path,
)

config.sample_count = 10

# Override the dataset settings to use your local file
config.dataset_path = dataset_path
config.dataset_name = None  # Clear dataset_name so it uses dataset_path instead

# Optionally update output path
config.output_path = "./analysis_output/banking77"

# IMPORTANT: Disable analyzers that require large model downloads or have issues
# IFD requires downloading Qwen model and may cause MPS crashes
# fasttext requires additional dependencies
# repr_diversity and question_diversity download embedding models
print(f"Running {len(config.analyzers)} analyzers: {[a.id for a in config.analyzers]}")

# Validate the configuration
config.finalize_and_validate()

# Create the analyzer
analyzer = DatasetAnalyzer(config)

Running 18 analyzers: ['length', 'diversity', 'format', 'quality', 'training_quality', 'cost', 'content_pattern', 'question_diversity', 'fasttext', 'ifd', 'repr_diversity', 'task_category', 'safety', 'difficulty', 'instruct_reward', 'input_quality', 'conversation_structure', 'response_completeness']
[2025-12-30 10:42:58,644][oumi][rank0][pid:36804][MainThread][INFO]][base_map_dataset.py:91] Creating map dataset (type: TextSftJsonLinesDataset)... dataset_name: 'custom'
[2025-12-30 10:42:58,645][oumi.utils.analysis_utils][rank0][pid:36804][MainThread][INFO]][analysis_utils.py:225] Loaded text dataset from: /Users/ryanarman/code/scratch/ryan_hillclimbing_experiments/banking77/notebooks/data/banking77_train.jsonl
[2025-12-30 10:42:58,645][oumi][rank0][pid:36804][MainThread][INFO]][dataset_analyzer.py:154] Loaded dataset from config: None
[2025-12-30 10:42:58,766][oumi][rank0][pid:36804][MainThread][INFO]][dataset_analyzer.py:304] Initialized sample analyzer: length
[2025-12-30 10:42:58,767

In [3]:
# Run the analysis
analyzer.analyze_dataset()

# The results are stored in analyzer object
if analyzer._analysis_results:
    print(
        f"Total conversations analyzed: {analyzer._analysis_results.conversations_analyzed}"
    )

[2025-12-30 10:42:58,798][oumi][rank0][pid:36804][MainThread][INFO]][dataset_analyzer.py:347] Starting analysis of dataset: None
[2025-12-30 10:42:58,799][oumi][rank0][pid:36804][MainThread][INFO]][dataset_analyzer.py:348] Using 18 sample analyzers: ['length', 'diversity', 'format', 'quality', 'training_quality', 'cost', 'content_pattern', 'question_diversity', 'fasttext', 'ifd', 'repr_diversity', 'task_category', 'safety', 'difficulty', 'instruct_reward', 'input_quality', 'conversation_structure', 'response_completeness']
[2025-12-30 10:42:58,800][oumi][rank0][pid:36804][MainThread][INFO]][dataset_analyzer.py:371] Analyzing 10 of 8002 conversations
[2025-12-30 10:42:58,800][oumi][rank0][pid:36804][MainThread][INFO]][dataset_analyzer.py:421] Converting conversation dataset with 8002 items
[2025-12-30 10:42:58,801][oumi][rank0][pid:36804][MainThread][INFO]][dataset_analyzer.py:428] Limiting analysis to first 10 items (dataset has 8002 total)


Converting Unknown Dataset to DataFrames: 100%|██████████| 10/10 [00:00<00:00, 1885.67item/s]


[2025-12-30 10:42:58,852][oumi][rank0][pid:36804][MainThread][INFO]][question_diversity_analyzer.py:464] Computing embeddings for 10 user questions...
[2025-12-30 10:42:58,853][oumi][rank0][pid:36804][MainThread][INFO]][question_diversity_analyzer.py:174] Loading embedding model: all-MiniLM-L6-v2


Computing embeddings: 100%|██████████| 10/10 [00:00<00:00, 95.92it/s]


[2025-12-30 10:43:00,056][oumi][rank0][pid:36804][MainThread][INFO]][question_diversity_analyzer.py:469] Clustering 10 questions using dbscan...
[2025-12-30 10:43:00,270][oumi][rank0][pid:36804][MainThread][INFO]][question_diversity_analyzer.py:487] Found 1 clusters
[2025-12-30 10:43:00,286][oumi][rank0][pid:36804][MainThread][INFO]][fasttext_analyzer.py:220] Initialized fast-langdetect for language detection
[2025-12-30 10:43:00,288][oumi][rank0][pid:36804][MainThread][INFO]][fasttext_analyzer.py:436] Analyzing language for column: conversation_text_content
[2025-12-30 10:43:00,428][oumi][rank0][pid:36804][MainThread][INFO]][ifd_analyzer.py:154] Loading model for IFD analysis: Qwen/Qwen3-0.6B
[2025-12-30 10:43:01,670][oumi][rank0][pid:36804][MainThread][INFO]][ifd_analyzer.py:193] Loaded Qwen/Qwen3-0.6B on cpu (dtype: torch.float32)
[2025-12-30 10:43:01,671][oumi][rank0][pid:36804][MainThread][INFO]][ifd_analyzer.py:664] Computing IFD scores using instruction='conversation_text_conten

...

[2025-12-30 10:43:04,735][oumi][rank0][pid:36804][MainThread][INFO]][question_diversity_analyzer.py:469] Clustering 10 questions using dbscan...
[2025-12-30 10:43:04,738][oumi][rank0][pid:36804][MainThread][INFO]][question_diversity_analyzer.py:482] Found 0 clusters, 10 unique/diverse questions (not similar to others)
[2025-12-30 10:43:04,739][oumi][rank0][pid:36804][MainThread][INFO]][fasttext_analyzer.py:436] Analyzing language for column: text_content
[2025-12-30 10:43:04,747][oumi][rank0][pid:36804][MainThread][INFO]][ifd_analyzer.py:643] Detected conversation format. Computing IFD for assistant messages using preceding user messages as instructions.
[2025-12-30 10:43:06,884][oumi][rank0][pid:36804][MainThread][INFO]][ifd_analyzer.py:521] IFD analysis complete. Processed 10 assistant messages out of 10 total.
[2025-12-30 10:43:06,885][oumi][rank0][pid:36804][MainThread][INFO]][repr_diversity_analyzer.py:363] Computing diversity scores for 30 samples in column 'text_content'...


Computing embeddings: 100%|██████████| 30/30 [00:00<00:00, 170.71it/s]


[2025-12-30 10:43:07,063][oumi][rank0][pid:36804][MainThread][INFO]][repr_diversity_analyzer.py:230] Computing nearest neighbor distances for 30 samples (k=5)...
[2025-12-30 10:43:07,066][oumi][rank0][pid:36804][MainThread][INFO]][repr_diversity_analyzer.py:556] Column 'text_content': 19/30 samples (63.3%) are redundant


/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/Users/ryanarman/miniconda3/envs/oumi/lib/python3.11/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


Total conversations analyzed: 10


In [11]:
schema = analyzer.get_schema()
print(f"len(schema): {len(schema)}")
merged_columns = analyzer.analysis_df.columns
print(f"len(merged_columns): {len(merged_columns)}")

len(schema): 184
len(merged_columns): 184


In [9]:
# analyzer.analysis_df.head()

# Conv level

In [6]:
conv_columns = analyzer.conversation_df.columns
row = analyzer.conversation_df.iloc[0]
col = conv_columns[44]
print(f"col: {col}")
print(f"schema[col]: {schema[col]}")
print(f"row[col]: {row[col]}")

col: conversation_text_content__question_diversity__is_concentrated
schema[col]: {'type': <ColumnType.BOOL: 'bool'>, 'content_type': <ContentType.BOOLEAN: 'boolean'>, 'description': 'Whether question is in a concentrated cluster (>threshold)'}
row[col]: True


In [7]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

analyzer_names = [a.id for a in config.analyzers]
conv_columns = analyzer.conversation_df.columns
row = analyzer.conversation_df.iloc[0]

# Then use it directly
analyzer_name = analyzer_names[4]
print(f"Analyzer: {analyzer_name}")

filtered_cols = filter_analyzer_columns(conv_columns, analyzer_id=analyzer_name)
info = parse_analyzer_column_name(filtered_cols[0])
print("\nInput:")
# print(f"source_column: {info.source_column}")
print(f"{row[info.source_column]}\n")

for col in filtered_cols:
    print(f"metric: {info.metric_name}")
    # print(f"type: {schema[col]['type']}")
    # print(f"content_type: {schema[col]['content_type']}")
    print(f"description: {schema[col]['description']}")
    print(f"value: {row[col]}")
    print("\n")


Analyzer: training_quality

Input:
SYSTEM: You are a banking intent classifier. Classify the user's query into one of  77 banking intents (output is a single integer ID).

IDs:

0: activate_my_card
1: age_limit
2: apple_pay_or_google_pay
3: atm_support
4: automatic_top_up
5: balance_not_updated_after_bank_transfer
6: balance_not_updated_after_cheque_or_cash_deposit
7: beneficiary_not_allowed
8: cancel_transfer
9: card_about_to_expire
10: card_acceptance
11: card_arrival
12: card_delivery_estimate
13: card_linking
14: card_not_working
15: card_payment_fee_charged
16: card_payment_not_recognised
17: card_payment_wrong_exchange_rate
18: card_swallowed
19: cash_withdrawal_charge
20: cash_withdrawal_not_recognised
21: change_pin
22: compromised_card
23: contactless_not_working
24: country_support
25: declined_card_payment
26: declined_cash_withdrawal
27: declined_transfer
28: direct_debit_payment_not_recognised
29: disposable_card_limits
30: edit_personal_details
31: exchange_charge
32: exc

# Message level

In [8]:
from oumi.core.analyze.column_utils import (
    filter_analyzer_columns,
    get_analyzer_columns_by_analyzer,
    parse_analyzer_column_name,
)

analyzer_names = [a.id for a in config.analyzers]
row = analyzer.message_df.iloc[2]
msg_columns = analyzer.message_df.columns

# Then use it directly
analyzer_name = analyzer_names[4]
print(f"Analyzer: {analyzer_name}")

filtered_cols = filter_analyzer_columns(msg_columns, analyzer_id=analyzer_name)
info = parse_analyzer_column_name(filtered_cols[0])

print(f"[{row['role']}]: {row[info.source_column]}\n")
print("\nInput:")
# print(f"source_column: {info.source_column}")

for col in filtered_cols:
    print(f"metric: {info.metric_name}")
    # print(f"type: {schema[col]['type']}")
    # print(f"content_type: {schema[col]['content_type']}")
    print(f"description: {schema[col]['description']}")
    print(f"value: {row[col]}")
    print("\n")


Analyzer: training_quality
[assistant]: 52


Input:
metric: response_completeness_score
description: Response completeness score (0.0 = incomplete, 1.0 = complete)
value: 0.0


metric: response_completeness_score
description: Whether response has proper sentence ending
value: False


metric: response_completeness_score
description: Whether response has clear structure
value: False


metric: response_completeness_score
description: Word count of response
value: 1.0


